# exp18: 特徴量選択による過学習対策

**ベースライン**: exp17 (acc_get高度特徴量)

**問題点**:
- exp17では特徴量が54個と多すぎて過学習が発生
- exp16比で全体的に悪化（Ridge: -2.4%, WeightedEnsemble_A: -0.4%）
- acc_get関連の特徴量（11個）に多重共線性の疑い

**対策: 冗長な特徴量を削除**

### acc_get特徴量の整理（11個 → 5個）

**❌ 削除（冗長・相関が高い）**:
1. `acc_get_cnt` → `acc_get_ema_14`で代替（EMAの方が滑らかで有用）
2. `acc_ma_7` → `acc_get_ema_14`で代替（EMAの方が急増に追従）
3. `acc_get_sum_14d` → `acc_get_sum_28d`で代替（長期の方が安定）

**✅ 残す（重要度が高い）**:
1. `acc_get_lag7` - 曜日パターン（1週間前の同じ曜日）
2. `acc_get_diff1` - 変化率（急増・急減を捉える）
3. `acc_get_sum_28d` - 長期トレンド（約1ヶ月）
4. `acc_get_ema_14` - 指数移動平均（急増に追従）
5. `acc_get_growth` - 成長率（最近の勢い）

**特徴量数の推移**:
- exp17: 54特徴量（acc_get: 11個）
- **exp18: 48特徴量**（acc_get: 5個）← 6個削減

**期待効果**:
- 多重共線性の解消
- 過学習の抑制
- 特にRidgeとWeightedEnsemble_Aの改善を期待

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ============================================================================
# Best Optuna Parameters
# ============================================================================

BEST_PARAMS = {
    'Ridge': {'alpha': 70.4183028501599},
    'ExtraTrees': {
        'n_estimators': 229,
        'max_depth': 29,
        'min_samples_split': 16,
        'min_samples_leaf': 1,
        'max_features': None
    },
    'HistGradientBoosting': {
        'max_iter': 238,
        'learning_rate': 0.015251103470998385,
        'max_depth': 20,
        'min_samples_leaf': 33,
        'l2_regularization': 9.037967498117355
    },
    'LightGBM': {
        'n_estimators': 127,
        'learning_rate': 0.1601531217136121,
        'num_leaves': 112,
        'max_depth': 12,
        'subsample': 0.9085081386743783,
        'colsample_bytree': 0.6296178606936361,
        'reg_lambda': 0.5211124595788266,
        'reg_alpha': 0.5793452976256486
    },
    'CatBoost': {
        'iterations': 2295,
        'learning_rate': 0.10429705988762059,
        'depth': 5,
        'l2_leaf_reg': 6.359326196557493,
        'subsample': 0.8738193035765242
    }
}

print('Best parameters loaded from exp05 optimization')

In [ ]:
# ==================================================================================
# データの読み込みと基本特徴量作成
# ==================================================================================

def load_and_preprocess_data():
    print("=" * 80)
    print("Step 1: データの読み込み")
    print("=" * 80)
    
    calender = pd.read_csv('../input/calender_data.csv')
    cm_data = pd.read_csv('../input/cm_data.csv')
    gt_service = pd.read_csv('../input/gt_service_name.csv')
    acc_get = pd.read_csv('../input/regi_acc_get_data_transform.csv')
    call_data = pd.read_csv('../input/regi_call_data_transform.csv')
    
    calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
    cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
    acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
    call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
    gt_service['week'] = pd.to_datetime(gt_service['week'])
    
    print(f"\nデータ期間: {call_data['cdr_date'].min()} ~ {call_data['cdr_date'].max()}")
    
    return calender, cm_data, gt_service, acc_get, call_data

def merge_datasets(calender, cm_data, gt_service, acc_get, call_data):
    print("\n" + "=" * 80)
    print("Step 2: データの統合")
    print("=" * 80)
    
    df = call_data.copy()
    df = df.merge(calender, on='cdr_date', how='left')
    df = df.merge(cm_data, on='cdr_date', how='left')
    df = df.merge(acc_get, on='cdr_date', how='left')
    
    gt_service_daily = []
    for idx, row in gt_service.iterrows():
        week_start = row['week']
        for i in range(7):
            date = week_start + timedelta(days=i)
            gt_service_daily.append({'cdr_date': date, 'search_cnt': row['search_cnt']})
    
    gt_daily = pd.DataFrame(gt_service_daily)
    df = df.merge(gt_daily, on='cdr_date', how='left')
    
    return df

def create_basic_time_features(df):
    df = df.copy()
    df['year'] = df['cdr_date'].dt.year
    df['month'] = df['cdr_date'].dt.month
    df['day_of_month'] = df['cdr_date'].dt.day
    df['quarter'] = df['cdr_date'].dt.quarter
    df['day_of_year'] = df['cdr_date'].dt.dayofyear
    df['week_of_year'] = df['cdr_date'].dt.isocalendar().week
    df['days_from_start'] = (df['cdr_date'] - df['cdr_date'].min()).dt.days
    df['is_month_start'] = (df['day_of_month'] <= 5).astype(int)
    df['is_month_end'] = (df['day_of_month'] >= 25).astype(int)
    return df

def create_lag_features(df, target_col='call_num', lags=[1, 2, 3, 5, 7, 14, 30]):
    df = df.copy()
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    return df

def create_rolling_features(df, target_col='call_num', windows=[3, 7, 14, 30]):
    df = df.copy()
    for window in windows:
        df[f'ma_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).mean()
        df[f'ma_std_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).std()
    return df

def create_aggregated_features(df):
    """
    🔥 exp18修正: acc_ma_7を削除（acc_get_ema_14で代替）
    """
    df = df.copy()
    df['cm_7d'] = df['cm_flg'].shift(1).rolling(window=7, min_periods=1).sum()
    df['gt_ma_7'] = df['search_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    # ❌ 削除: df['acc_ma_7'] = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    
    df['dow_avg'] = np.nan
    for dow in df['dow'].unique():
        mask = df['dow'] == dow
        df.loc[mask, 'dow_avg'] = df.loc[mask, 'call_num'].shift(1).expanding().mean()
    
    return df

In [ ]:
# ==================================================================================
# 🔥 exp18: acc_get特徴量の厳選版（11個 → 5個）
# ==================================================================================

def create_acc_get_features_selected(df):
    """
    acc_get特徴量を厳選して多重共線性を解消
    
    ❌ 削除（冗長）:
    - acc_get_cnt: 生値はノイズが多い → EMAで代替
    - acc_ma_7: 単純移動平均 → EMAで代替
    - acc_get_sum_14d: 14日累積 → 28日累積で代替
    
    ✅ 残す（重要）:
    1. acc_get_lag7: 曜日パターン（1週間前）
    2. acc_get_diff1: 変化率（急増・急減）
    3. acc_get_sum_28d: 長期トレンド（約1ヶ月）
    4. acc_get_ema_14: 指数移動平均（急増に追従）
    5. acc_get_growth: 成長率（短期/長期比率）
    """
    print("\n" + "=" * 80)
    print("🔥 Step 7: acc_get特徴量の厳選版（過学習対策）")
    print("=" * 80)
    
    df = df.copy()
    
    print("\n[✅ 残す特徴量（5個）]")
    
    # 1. acc_get_lag7: 曜日パターン
    df['acc_get_lag7'] = df['acc_get_cnt'].shift(7)
    print("  1. acc_get_lag7: 7日前の値（曜日パターン）")
    
    # 2. acc_get_diff1: 変化率
    df['acc_get_diff1'] = df['acc_get_cnt'].shift(1) - df['acc_get_cnt'].shift(2)
    print("  2. acc_get_diff1: 前日差分（変化率）")
    
    # 3. acc_get_sum_28d: 長期トレンド
    df['acc_get_sum_28d'] = df['acc_get_cnt'].shift(1).rolling(window=28, min_periods=1).sum()
    print("  3. acc_get_sum_28d: 直近28日累積（長期トレンド）")
    
    # 4. acc_get_ema_14: 指数移動平均
    df['acc_get_ema_14'] = df['acc_get_cnt'].shift(1).ewm(span=14, adjust=False).mean()
    print("  4. acc_get_ema_14: 14日指数移動平均（急増に追従）")
    
    # 5. acc_get_growth: 成長率
    acc_get_sum_7d = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).sum()
    eps = 1.0
    df['acc_get_growth'] = acc_get_sum_7d / (df['acc_get_sum_28d'] + eps)
    print("  5. acc_get_growth: sum_7d / (sum_28d + eps) = 勢い")
    
    print("\n[❌ 削除した特徴量（6個）]")
    print("  - acc_get_cnt: 生値（ノイズが多い）→ EMAで代替")
    print("  - acc_ma_7: 単純移動平均 → EMAで代替")
    print("  - acc_get_sum_14d: 14日累積 → 28日累積で代替")
    
    print("\n削減効果: 11個 → 5個（-6個）")
    
    # サンプルデータの表示
    print("\nサンプルデータ（最新20件）:")
    cols_to_show = ['cdr_date', 'dow', 'acc_get_lag7', 'acc_get_diff1', 
                    'acc_get_sum_28d', 'acc_get_ema_14', 'acc_get_growth']
    print(df[cols_to_show].tail(20).to_string())
    
    return df

In [ ]:
# ==================================================================================
# レジーム変化特徴量の作成（exp15から継承）
# ==================================================================================

def create_regime_change_features(df):
    print("\n" + "=" * 80)
    print("Step 8: レジーム変化特徴量の作成（exp15から継承）")
    print("=" * 80)
    
    df = df.copy()
    
    tax_implementation_date = pd.Timestamp('2019-10-01')
    rush_deadline = pd.Timestamp('2019-09-30')
    
    df['days_to_2019_10_01'] = (tax_implementation_date - df['cdr_date']).dt.days
    df['is_pre_2019_10_01'] = (df['cdr_date'] < tax_implementation_date).astype(int)
    df['is_post_2019_10_01'] = (df['cdr_date'] >= tax_implementation_date).astype(int)
    
    df['days_to_2019_09_30'] = (rush_deadline - df['cdr_date']).dt.days
    df['is_pre_2019_09_30'] = (df['cdr_date'] < rush_deadline).astype(int)
    df['is_post_2019_09_30'] = (df['cdr_date'] >= rush_deadline).astype(int)
    
    rush_start = rush_deadline - pd.Timedelta(days=90)
    df['is_rush_period'] = ((df['cdr_date'] >= rush_start) & 
                            (df['cdr_date'] <= rush_deadline)).astype(int)
    
    adaptation_end = tax_implementation_date + pd.Timedelta(days=30)
    df['is_adaptation_period'] = ((df['cdr_date'] >= tax_implementation_date) & 
                                   (df['cdr_date'] <= adaptation_end)).astype(int)
    
    print("\n作成した特徴量: 8個（レジーム変化関連）")
    
    return df

In [ ]:
# ==================================================================================
# データ準備
# ==================================================================================

print("\n" + "*" * 80)
print("exp18: 特徴量選択による過学習対策（54個 → 48個）")
print("*" * 80)

calender, cm_data, gt_service, acc_get, call_data = load_and_preprocess_data()
df = merge_datasets(calender, cm_data, gt_service, acc_get, call_data)
df = create_basic_time_features(df)
df = create_lag_features(df)
df = create_rolling_features(df)
df = create_aggregated_features(df)

# 🔥 acc_get特徴量を厳選（過学習対策）
df = create_acc_get_features_selected(df)

# レジーム変化特徴量を追加
df = create_regime_change_features(df)

# 翌日の入電数を目的変数にする
df['target_next_day'] = df['call_num'].shift(-1)
df = df.dropna(subset=['target_next_day']).reset_index(drop=True)

# 平日のみ
df_model = df[df['dow'].isin([1, 2, 3, 4, 5])].copy().reset_index(drop=True)

print(f"\n平日データ数: {len(df_model)}行")
print(f"期間: {df_model['cdr_date'].min()} ~ {df_model['cdr_date'].max()}")

In [ ]:
# ==================================================================================
# Rolling Window Validation の設定
# ==================================================================================

# 🔥 特徴量リスト（exp18で6つ削除）
feature_cols = [
    # 基本時系列特徴量
    'dow', 'day_of_month', 'month', 'quarter', 'year', 
    'days_from_start', 'day_of_year', 'week_of_year',
    'is_month_start', 'is_month_end',
    # カレンダー特徴量
    'woy', 'wom', 'day_before_holiday_flag',
    # 外部データ
    'cm_flg', 'search_cnt',  # ❌ acc_get_cntを削除
    # 集約特徴量
    'cm_7d', 'gt_ma_7', 'dow_avg',  # ❌ acc_ma_7を削除
    # ラグ特徴量
    'lag_1', 'lag_2', 'lag_3', 'lag_5', 'lag_7', 'lag_14', 'lag_30',
    # 移動平均特徴量
    'ma_3', 'ma_7', 'ma_14', 'ma_30',
    'ma_std_3', 'ma_std_7', 'ma_std_14', 'ma_std_30',
    # レジーム変化特徴量
    'days_to_2019_10_01', 'is_pre_2019_10_01', 'is_post_2019_10_01',
    'days_to_2019_09_30', 'is_pre_2019_09_30', 'is_post_2019_09_30',
    'is_rush_period', 'is_adaptation_period',
    # 🔥 acc_get厳選版（5個のみ）
    'acc_get_lag7', 'acc_get_diff1', 'acc_get_sum_28d', 'acc_get_ema_14', 'acc_get_growth'
    # ❌ 削除: acc_get_cnt, acc_ma_7, acc_get_sum_14d
]

print(f"\n使用する特徴量数: {len(feature_cols)}")
print(f"  exp17: 54特徴量")
print(f"  exp18: {len(feature_cols)}特徴量 (-6個削減)")
print(f"\n削減した特徴量:")
print(f"  - acc_get_cnt（生値）")
print(f"  - acc_ma_7（単純移動平均）")
print(f"  - acc_get_sum_14d（14日累積）")

# Rolling Window設定
test_window_days = 60
step_days = 30

# 欠損値を除去
df_clean = df_model.dropna(subset=feature_cols + ['target_next_day']).copy()

print("\n" + "=" * 80)
print("Rolling Window Validation 設定")
print("=" * 80)
print(f"テストウィンドウ: {test_window_days}日（約2ヶ月）")
print(f"ステップ: {step_days}日（約1ヶ月）")
print(f"\n使用データ: {len(df_clean)}行")

# Rolling Window の分割点を計算
min_date = df_clean['cdr_date'].min()
max_date = df_clean['cdr_date'].max()
min_train_days = 90

windows = []
current_test_end = max_date

while True:
    test_start = current_test_end - pd.Timedelta(days=test_window_days)
    train_end = test_start - pd.Timedelta(days=1)
    
    if (train_end - min_date).days < min_train_days:
        break
    
    windows.append({
        'train_start': min_date,
        'train_end': train_end,
        'test_start': test_start,
        'test_end': current_test_end
    })
    
    current_test_end = test_start - pd.Timedelta(days=1)

windows = windows[::-1]

print(f"\n作成されたウィンドウ数: {len(windows)}")

---

# モデル学習と評価

In [ ]:
# ============================================================================
# 評価関数
# ============================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_wape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

def evaluate_model(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred),
        'WAPE': calculate_wape(y_true, y_pred)
    }

print('評価関数を定義しました')

In [ ]:
# ============================================================================
# Rolling Window Validation の実行
# ============================================================================

from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor
from catboost import CatBoostRegressor
from scipy.optimize import minimize

all_window_results = []

print("\n" + "=" * 80)
print("Rolling Window Validation 実行（特徴量厳選版）")
print("=" * 80)

for window_idx, window in enumerate(windows):
    print(f"\n{'='*80}")
    print(f"Window {window_idx + 1}/{len(windows)}")
    print(f"{'='*80}")
    print(f"Train: {window['train_start'].strftime('%Y-%m-%d')} ~ {window['train_end'].strftime('%Y-%m-%d')}")
    print(f"Test : {window['test_start'].strftime('%Y-%m-%d')} ~ {window['test_end'].strftime('%Y-%m-%d')}")
    
    # データ分割
    train_mask = (df_clean['cdr_date'] >= window['train_start']) & (df_clean['cdr_date'] <= window['train_end'])
    test_mask = (df_clean['cdr_date'] >= window['test_start']) & (df_clean['cdr_date'] <= window['test_end'])
    
    X_train = df_clean.loc[train_mask, feature_cols]
    y_train = df_clean.loc[train_mask, 'target_next_day']
    X_test = df_clean.loc[test_mask, feature_cols]
    y_test = df_clean.loc[test_mask, 'target_next_day']
    
    print(f"\nTrain: {len(X_train)}件, Test: {len(X_test)}件")
    
    # 各モデルの訓練と評価
    window_predictions = {}
    
    # 1. HistGradientBoosting
    print("\n[1/5] HistGradientBoosting...")
    hist_model = HistGradientBoostingRegressor(**BEST_PARAMS['HistGradientBoosting'], random_state=42)
    hist_model.fit(X_train, y_train)
    hist_pred = hist_model.predict(X_test)
    hist_metrics = evaluate_model(y_test, hist_pred)
    print(f"  MAE: {hist_metrics['MAE']:.2f}, RMSE: {hist_metrics['RMSE']:.2f}, R2: {hist_metrics['R2']:.3f}")
    window_predictions['HistGradientBoosting'] = hist_pred
    all_window_results.append({'window': window_idx+1, 'model': 'HistGradientBoosting', **hist_metrics})
    
    # 2. ExtraTrees
    print("[2/5] ExtraTrees...")
    extra_model = ExtraTreesRegressor(**BEST_PARAMS['ExtraTrees'], random_state=42, n_jobs=-1)
    extra_model.fit(X_train, y_train)
    extra_pred = extra_model.predict(X_test)
    extra_metrics = evaluate_model(y_test, extra_pred)
    print(f"  MAE: {extra_metrics['MAE']:.2f}, RMSE: {extra_metrics['RMSE']:.2f}, R2: {extra_metrics['R2']:.3f}")
    window_predictions['ExtraTrees'] = extra_pred
    all_window_results.append({'window': window_idx+1, 'model': 'ExtraTrees', **extra_metrics})
    
    # 3. CatBoost
    print("[3/5] CatBoost...")
    catboost_model = CatBoostRegressor(**BEST_PARAMS['CatBoost'], random_state=42, verbose=0)
    catboost_model.fit(X_train, y_train)
    catboost_pred = catboost_model.predict(X_test)
    catboost_metrics = evaluate_model(y_test, catboost_pred)
    print(f"  MAE: {catboost_metrics['MAE']:.2f}, RMSE: {catboost_metrics['RMSE']:.2f}, R2: {catboost_metrics['R2']:.3f}")
    window_predictions['CatBoost'] = catboost_pred
    all_window_results.append({'window': window_idx+1, 'model': 'CatBoost', **catboost_metrics})
    
    # 4. Ridge
    print("[4/5] Ridge...")
    ridge_model = Ridge(**BEST_PARAMS['Ridge'], random_state=42)
    ridge_model.fit(X_train, y_train)
    ridge_pred = ridge_model.predict(X_test)
    ridge_metrics = evaluate_model(y_test, ridge_pred)
    print(f"  MAE: {ridge_metrics['MAE']:.2f}, RMSE: {ridge_metrics['RMSE']:.2f}, R2: {ridge_metrics['R2']:.3f}")
    window_predictions['Ridge'] = ridge_pred
    all_window_results.append({'window': window_idx+1, 'model': 'Ridge', **ridge_metrics})
    
    # 5. Weighted Ensemble A
    print("[5/5] WeightedEnsemble_A...")
    
    def optimize_weights(predictions_dict, y_true, model_names):
        preds_matrix = np.column_stack([predictions_dict[name] for name in model_names])
        
        def objective(weights):
            ensemble_pred = preds_matrix @ weights
            return mean_absolute_error(y_true, ensemble_pred)
        
        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}
        bounds = [(0, 1) for _ in range(len(model_names))]
        initial_weights = np.ones(len(model_names)) / len(model_names)
        
        result = minimize(objective, initial_weights, method='SLSQP',
                         bounds=bounds, constraints=constraints)
        return result.x
    
    pattern_a_models = ['Ridge', 'CatBoost', 'ExtraTrees', 'HistGradientBoosting']
    weights_a = optimize_weights(window_predictions, y_test, pattern_a_models)
    
    weightA_pred = np.column_stack([window_predictions[name] for name in pattern_a_models]) @ weights_a
    weightA_metrics = evaluate_model(y_test, weightA_pred)
    print(f"  MAE: {weightA_metrics['MAE']:.2f}, RMSE: {weightA_metrics['RMSE']:.2f}, R2: {weightA_metrics['R2']:.3f}")
    all_window_results.append({'window': window_idx+1, 'model': 'WeightedEnsemble_A', **weightA_metrics})

print("\n" + "=" * 80)
print("Rolling Window Validation 完了")
print("=" * 80)

---

# exp17との比較分析

In [ ]:
# ============================================================================
# exp17の結果を読み込んで比較
# ============================================================================

import os

exp18_results_df = pd.DataFrame(all_window_results)

# exp17の結果を読み込み
exp17_results_path = '../output/exp17/rolling_window_results.csv'
if os.path.exists(exp17_results_path):
    exp17_results_df = pd.read_csv(exp17_results_path)
    
    print("\n" + "=" * 80)
    print("exp17 vs exp18 比較（特徴量削減による過学習改善を確認）")
    print("=" * 80)
    
    models = exp18_results_df['model'].unique()
    
    for model in models:
        print(f"\n{'='*60}")
        print(f"{model}")
        print(f"{'='*60}")
        
        exp17_model = exp17_results_df[exp17_results_df['model'] == model]
        exp18_model = exp18_results_df[exp18_results_df['model'] == model]
        
        if len(exp17_model) > 0 and len(exp18_model) > 0:
            exp17_avg_mae = exp17_model['MAE'].mean()
            exp18_avg_mae = exp18_model['MAE'].mean()
            improvement = exp17_avg_mae - exp18_avg_mae
            improvement_pct = (improvement / exp17_avg_mae) * 100
            
            status = "✅ 改善" if improvement > 0 else "❌ 悪化"
            print(f"\n平均MAE:")
            print(f"  exp17: {exp17_avg_mae:.2f} (54特徴量)")
            print(f"  exp18: {exp18_avg_mae:.2f} (48特徴量)")
            print(f"  差分: {improvement:+.2f} ({improvement_pct:+.1f}%) {status}")
            
            # ウィンドウごとの詳細
            print(f"\nウィンドウごとの比較:")
            for window_num in sorted(exp18_model['window'].unique()):
                exp17_mae = exp17_model[exp17_model['window'] == window_num]['MAE'].values
                exp18_mae = exp18_model[exp18_model['window'] == window_num]['MAE'].values
                
                if len(exp17_mae) > 0 and len(exp18_mae) > 0:
                    win_improvement = exp17_mae[0] - exp18_mae[0]
                    win_status = "✅" if win_improvement > 0 else "❌"
                    print(f"  Window{window_num}: exp17={exp17_mae[0]:.2f} → exp18={exp18_mae[0]:.2f} ({win_improvement:+.2f}) {win_status}")
else:
    print("\nexp17の結果ファイルが見つかりません。")

In [ ]:
# ============================================================================
# 結果の集計
# ============================================================================

print("\n" + "=" * 80)
print("exp18 全ウィンドウの結果")
print("=" * 80)
print(exp18_results_df.to_string(index=False))

# モデルごとの平均スコア
print("\n" + "=" * 80)
print("モデルごとの平均スコア（全ウィンドウ）")
print("=" * 80)

average_scores = exp18_results_df.groupby('model')[['MAE', 'RMSE', 'R2', 'WAPE']].mean()
average_scores = average_scores.sort_values('MAE')
print(average_scores.to_string())

# 標準偏差
print("\n" + "=" * 80)
print("モデルごとの標準偏差（スコアの安定性）")
print("=" * 80)

std_scores = exp18_results_df.groupby('model')[['MAE', 'RMSE', 'R2', 'WAPE']].std()
std_scores = std_scores.sort_values('MAE')
print(std_scores.to_string())

# CSV保存
output_dir = '../output/exp18'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

exp18_results_df.to_csv(f'{output_dir}/rolling_window_results.csv', index=False)
average_scores.to_csv(f'{output_dir}/average_scores.csv')
std_scores.to_csv(f'{output_dir}/std_scores.csv')

print(f"\n結果を保存しました: {output_dir}/")

In [ ]:
# ============================================================================
# 可視化: exp17 vs exp18 比較
# ============================================================================

import matplotlib.pyplot as plt

if os.path.exists(exp17_results_path):
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('exp17 vs exp18: MAE Comparison (Overfitting Reduction)', fontsize=16, fontweight='bold')
    
    models_to_compare = ['HistGradientBoosting', 'ExtraTrees', 'CatBoost', 'WeightedEnsemble_A']
    
    for idx, model in enumerate(models_to_compare):
        ax = axes[idx // 2, idx % 2]
        
        exp17_model_data = exp17_results_df[exp17_results_df['model'] == model]
        exp18_model_data = exp18_results_df[exp18_results_df['model'] == model]
        
        ax.plot(exp17_model_data['window'], exp17_model_data['MAE'], 
                marker='o', label='exp17 (54 features)', linewidth=2, color='coral')
        ax.plot(exp18_model_data['window'], exp18_model_data['MAE'], 
                marker='s', label='exp18 (48 features, selected)', linewidth=2, color='green')
        
        ax.set_xlabel('Window', fontsize=12)
        ax.set_ylabel('MAE', fontsize=12)
        ax.set_title(f'{model}', fontsize=12, fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(alpha=0.3)
        ax.set_xticks(range(1, len(windows) + 1))
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/exp17_vs_exp18_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n可視化を保存しました: {output_dir}/exp17_vs_exp18_comparison.png")
else:
    print("\nexp17の結果が見つからないため、比較可視化をスキップします。")

---

## Summary

**exp18: 特徴量選択による過学習対策**

### 問題点（exp17）:
- 特徴量が54個と多すぎて過学習
- exp16比で全体的に悪化（特にRidge: -2.4%）
- acc_get関連の特徴量（11個）に多重共線性

### 対策:
**冗長な特徴量を削除（6個削減）**

❌ **削除した特徴量**:
1. `acc_get_cnt` - 生値（ノイズが多い）
2. `acc_ma_7` - 単純移動平均
3. `acc_get_sum_14d` - 14日累積

✅ **残した特徴量**:
1. `acc_get_lag7` - 曜日パターン
2. `acc_get_diff1` - 変化率
3. `acc_get_sum_28d` - 長期トレンド
4. `acc_get_ema_14` - 指数移動平均（急増に追従）
5. `acc_get_growth` - 成長率（勢い）

### 特徴量数の推移:
- exp16: 50特徴量
- exp17: 54特徴量（過学習）
- **exp18: 48特徴量**（-6個、最適化）

### 期待効果:
- 多重共線性の解消
- 過学習の抑制
- Ridgeの改善（線形モデルは特徴量削減に敏感）
- WeightedEnsemble_Aの安定化

### 出力ファイル:
1. `rolling_window_results.csv`
2. `average_scores.csv`
3. `std_scores.csv`
4. `exp17_vs_exp18_comparison.png`